# AI Project 2
## Arman Rostami

In this project we'll try to design an agent to play a game like Checkers using Minimax algorithm. Next, we'll use Alpha–beta pruning to improve speed by decreasing the number of nodes that are going to be evaluated.

## Minimax

Minimax algorithm is used in Aversarial search problems where there are two agents competing each other and each agent tries to plan ahead based on the world and other agent actions.

### Agents

There are two types of Agents in Minimax algorithm : Min agent and Max agent.

Min agent is the agent which tries to minimize other agent's value by selecting the best move possible, Max agent tries to maximize it's value by selecting the best move respectively.

### Algorithm

Minimax tries to find the best move for agent which may maximize it's value or minimize other agent's value by selecting a move based on the state's successor's values which may be maximum move or minimum move based on agent's type.

It computes game tree by generating each state's successor states using a recursive approach. By using a complete depth-first exploration of the game tree, if the maximum depth of the tree is m and there are b legal moves at each point, then the time complexity of the minimax algorithm is $O(b ^ m)$. The space complexity is $O(bm)$ for an
algorithm that generates all actions at once, or $O(m)$ for an algorithm that generates actions one at a time.

Based on type of the agent's node in game tree, node's value is the best reachable value from expanding node to reach the leaves. leaves values correspond to one agent's value which can be evaluated by an utility function which assigns an value to leaf as one agent's score.

### Evaluation Function

Game tree can be very large and may be difficult to expand whole game tree to reach leaves and find nodes values because of time limits or resource limits. One solution is to use an evaluation function to evaluate values for non-terminal nodes.

We'll search to a limited depth by using Depth-limited search. When reaching a non-terminal node, value of evaluation function of that node is returned as it's value.

The evaluation function chosen in this game uses two other evaluation functions to evaluate nodes values. It uses each evaluation function with a uniform weight.

First evalution function computes number of Max agent player and Min agent player possible moves and returns it's difference. Having more possible moves may increase chance of winning.

Second evaluation function examines number of players pieces. It returns difference between Max agent pieces and Min agent pieces. A state with more agent's pieces is a better state for that agent.

### Implementation

The code below shows initialize function of MinimaxPlayer class which sets remaining search depth and board and agent's side to also represent the current state of player.

In [9]:
def initialize(self, side, depth, board = None):
    self.side = side
    self.depth = depth
    self.name = "Minimax"
    if board != None:
        self.board = copy.deepcopy(board)

We'll use the following codes in MinimaxPlayer class to calculate evalution function. $\texttt{getEvaluation1}$ is the first evaluation function and $\texttt{getEvaluation2}$ is the second evaluation function which are used by $\texttt{getEvaluation}$ function with a uniform weight.

In [3]:
def getEvaluation1(self, board):
    bMoves = self.generateMoves(board, 'B')
    wMoves = self.generateMoves(board, 'W')
    return len(bMoves) - len(wMoves)

In [7]:
def getEvaluation2(self, board):
    bPieces = 0
    wPieces = 0
    for row in board:
        for piece in row:
            if piece == 'B':
                bPieces += 1
            elif piece == 'W':
                wPieces += 1
    return bPieces - wPieces

In [8]:
def getEvaluation(self, board):
    return self.getEvaluation1(board) + self.getEvaluation2(board)

The following code shows implementation of $\texttt{getMove}$ function which returns best move for Minimax player based on it's type. If agent is the Max agent, It selects a move which makes the maximum value otherwise it returns best move which makes the minimum value of other agent. It uses $\texttt{getValue}$ function to get value of successor nodes values.

In [10]:
def getMove(self, board):
    moves = self.generateMoves(board, self.side)
    values = []
    for move in moves:
        child = self.createChild(board)
        child.makeMove(self.side, move)
        childValue = child.getValue(child.board)
        values.append(childValue)
    if len(moves) == 0:
        return []
    elif self.side == 'B':
        return moves[values.index(max(values))]
    elif self.side == 'W':
        return moves[values.index(min(values))]
    else:
        raise GameError

The code below shows implementation of $\texttt{getValue}$ function which returns node's value. If current node is Max agent's turn, It returns maximum value of current node's successor's values which are other agent's turn who tries to minimize Max agent's value. On Min agent's turn, It returns minimum value of current node's successors which are other agent's turn who tries to maximize it's value. When reaching a win or lose state or a state with zero depth, value of $\texttt{getEvaluation}$ for that node is returned. This function uses $\texttt{createChild}$ function to generate a player's successor state.

In [11]:
def getValue(self, board):
    if self.depth == 0:
        return self.getEvaluation(board)
    moves = self.generateMoves(board, self.side)
    if len(moves) == 0:
        return self.getEvaluation(board)
    value = -math.inf if self.side == 'B' else math.inf
    for move in moves:
        child = self.createChild(board)
        child.makeMove(self.side, move)
        if self.side == 'B':
            value = max(value, child.getValue(child.board))
        else:
            value = min(value, child.getValue(child.board))
    return value

The following code shows implementation of $\texttt{createChild}$ function.

In [12]:
def createChild(self, board):
    child = MinimaxPlayer(self.size)
    childSide = self.opponent(self.side)
    child.initialize(childSide, self.depth - 1, board)
    return child

## Pruning

Pruning is an algorithm used to reduce the number of nodes that have to be evaluated to reach the optimal solution. Pruning works by blocking the evaluation of nodes whose leaf nodes would give worse results compared to the one that was previously examined. We'll use Alpha-beta pruning algorithm to prune Minimax tree.

### Alpha-beta Pruning

Alpha-beta pruning tries to decrease number of nodes evaluated by Minimax algorithm using two parameters: alpha and beta.

alpha corresponds to Max agent's best option on path to root and beta corresponds to Min agent's best option on path to root respectively.

Max agent tries to find the maximum value from it's node successors which are Min agents. By having the best Max option so far as alpha, expanding nodes with value equal or less than alpha is not necessary because it won't be selected by Max agent and since Min agent tries to find minimum value of it's node successors, expanding can be stopped. Same logic goes to Min agent which tries to find the minimum value from it's node successors. By having the best value of Min agent's node as beta, we can stop expanding nodes on Max node with value equal or more than beta because this node won't be seleted because there's a better node which can be selected by the Min agent.

Alpha-beta pruning selectes the same moves as Minimax player except in equal value states which alpha-beta selectes the first equal node if other successors are pruned. This property depends on implementation of Alpha-beta pruning.

### Alpha-beta Pruning Implementation

AlphaBetaPlayer class is implemented to run Minimax with Alpha-beta pruning. It almost has same functions as MinimaxPlayer class except $\texttt{getMove}$ and $\texttt{getValue}$ functions which are modified to support Alpha-beta pruning.

The code below shows implementation of AlphaBetaPlayer's $\texttt{getMove}$ function. This function updates alpha and beta based on Agent's type and calls getValue with alpha and beta.

In [15]:
def getMove(self, board):
    alpha = -math.inf
    beta = math.inf
    moves = self.generateMoves(board, self.side)
    values = []
    for move in moves:
        child = self.createChild(board)
        child.makeMove(self.side, move)
        childValue = child.getValue(child.board, alpha, beta)
        if self.side == 'B':
            alpha = max(alpha, childValue)
        else:
            beta = min(beta, childValue)
        values.append(childValue)
    if len(moves) == 0:
        return []
    elif self.side == 'B':
        return moves[values.index(max(values))]
    elif self.side == 'W':
        return moves[values.index(min(values))]
    else:
        raise GameError

The following code shows implementation of $\texttt{getValue}$ function. This function prune unnecessary nodes by considering agent's type and alpha and beta value.

In [16]:
def getValue(self, board, alpha, beta):
    if self.depth == 0:
        return self.getEvaluation(board)
    moves = self.generateMoves(board, self.side)
    if len(moves) == 0:
        return self.getEvaluation(board)
    value = -math.inf if self.side == 'B' else math.inf
    for move in moves:
        child = self.createChild(board)
        child.makeMove(self.side, move)
        if self.side == 'B':
            value = max(value, child.getValue(child.board, alpha, beta))
            if value >= beta:
                return value
            alpha = max(alpha, value)
        else:
            value = min(value, child.getValue(child.board, alpha, beta))
            if value <= alpha:
                return value
            beta = min(beta, value)
    return value

## Results

Based on tests time and agent's winnings the best option for depth in MinimaxPlayer is chosen to be 3 and on AlphaBetaPlayer it's 4. The following table shows result for 5 games.

| First Player    | Second Player | First Player Depth | First Player winings | Average First Player Move Selection Time (s)|
| :----------:    | :-----------: | :----------------: | :------------------: | :------------------------------: |
| MinimaxPlayer   | Random Player | 3                  | 5 / 5                | 0.0370                           |
| AlphaBetaPlayer | Random Player | 3                  | 5 / 5                | 0.0198                           |
| MinimaxPlayer   | Random Player | 4                  | 5 / 5                | 0.3953                           |
| AlphaBetaPlayer | Random Player | 4                  | 5 / 5                | 0.0739                           |